In [1]:
# create a Jupyter image that will be our display surface
# format can be 'jpeg' or 'png'; specify width and height to set viewer size
# PNG will be a little clearer, especially with overlaid graphics, but
# JPEG is faster to update
import ipywidgets as widgets
from glowing_waffles.differential_photometry import catalog_search, in_frame
from glowing_waffles.differential_photometry  import filter_transform
from glowing_waffles.io import parse_aij_table
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord
import numpy as np

In [2]:
jup_img = widgets.Image(format='jpeg', width=500, height=500)

In [3]:
# Boilerplate to create a Ginga viewer connected to this widget
# this could be simplified, or hidden behind a class or convenience 
# method
# NOTE: you don't have to specify a log file--and if you are not interested
# in the log just specify null=True for a null logger
# level=10 will give you the most debugging information
from ginga.misc.log import get_logger
logger = get_logger("my viewer", log_stderr=False, log_file='/tmp/ginga.log', level=40)

from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView
v1 = EnhancedCanvasView(logger=logger)
# set our linkage between the jupyter widget at ginga
v1.set_widget(jup_img)

# enable all possible keyboard and pointer operations
bd = v1.get_bindings()
bd.enable_all(True)

In [4]:
coordinates = widgets.HTML('<h3>coordinates show up here</h3>')

# callback to display position in RA/DEC deg
def mouse_move(viewer, button, data_x, data_y, w):
    image = viewer.get_image()
    if image is not None:
        ra, dec = image.pixtoradec(data_x, data_y)
        w.value = "RA: %f, Dec: %f" % (ra, dec)

v1.add_callback('motion', mouse_move, coordinates)


In [5]:
v1.load_fits("/Users/mcraig/Development/astronomy/Glowing-Pancakes/TrES-3b-004R.fit")
img = v1.get_image()

In [6]:
#Use glowing waffles to parse the measurements file
aij_stars = parse_aij_table('tres_3_measurements_4.csv')

#get the ra (which aij gives in hour angle) from the raw aij data
aij_ra = np.array([star.ra.mean().value for star in aij_stars])
#get the dec from the raw aij data
aij_dec = [star.dec.mean() for star in aij_stars]
x, y = img.wcs.wcs.all_world2pix(aij_ra, aij_dec, 0)
aij_name = range(1, len(aij_stars) + 1)
aij_cat = Table([
    Column(data=aij_name, name='Name'),
    Column(data=x, name='x'),
    Column(data=y, name='y')
])

In [7]:
apass_cat, x, y = catalog_search(img.wcs.wcs, img.shape, 'II/336/apass9', radius=1, padding=0)
apass_cat.add_column(Column(data=x, name='x'))
apass_cat.add_column(Column(data=y, name='y'))

In [8]:
vsx_cat, x, y = catalog_search(img.wcs.wcs, img.shape, 'B/vsx/vsx', radius=1, padding=0)
vsx_cat.add_column(Column(data=x, name='x'))
vsx_cat.add_column(Column(data=y, name='y'))

cats = {
    'VSX': vsx_cat,
    'APASS': apass_cat,
    'AIJ Index': aij_cat
}

In [9]:
catalog = widgets.SelectMultiple(description='Catalog', options=['None', 'VSX', 'APASS', 'AIJ Index'])
                           #options=[('None', None), 
                           #         ('VSX', 'vsx_cat'), 
                           #         ('APASS', apass_cat),
                           #         ('AIJ Index', aij_cat)])

In [10]:
# add a canvas to the image and set the draw type
canvas = v1.add_canvas()

In [11]:
catalog.value

()

In [12]:
detail = widgets.HTML('Click object for details from selected catalog')

def mouse_click(viewer, button, data_x, data_y, w):
    global catalog
    w.value = 'shazam!'
    img = viewer.get_image()
    ra, dec = img.pixtoradec(data_x, data_y)
    coord = SkyCoord(ra, dec, unit='deg')
    w.value = '%f %f' % (ra, dec)
    #w.value = catalog.value[0]
    if catalog.value[0] == 'None':
        w.value = ''
        return
    cat = cats[catalog.value[0]]
    cat_coords = SkyCoord(cat['RAJ2000'], cat['DEJ2000'])
    idx, d2d, d3d = coord.match_to_catalog_sky(cat_coords)
    content = []
    for c in cat.colnames:
        if not cat[c][idx]:
            continue
        val = '<strong>{}</strong> ({}): {}'.format(c, cat[c].unit, cat[c][idx])
        content.append(val)
    w.value = '<br>'.join(content)

v1.add_callback('cursor-up', mouse_click, detail)

In [13]:
# embed the viewer here

viewer = widgets.VBox([coordinates, catalog, detail])
jup_img.layout.width = jup_img.width
jup_img.layout.height = jup_img.height
img_box = widgets.VBox([jup_img])
img_box.layout.width = jup_img.width
img_box.layout.height = jup_img.height
hb = widgets.HBox([img_box, viewer])

In [14]:
def overlay_catalog(change):
    catalogs = change['new']
    img = v1.get_image()
    canvas.delete_all_objects()
    colors = ['yellow', 'cyan', 'red']
    offsets = [0, -20, -40]
    for new_catalog, color, offset in zip(catalogs, colors, offsets):
        if new_catalog is 'None':
            return
        else:
            cat = cats[new_catalog]
        # Draw circles around all objects
        Circle = canvas.get_draw_class('circle')
        Text = canvas.get_draw_class('text')
        try:
            names = cat['Name']
        except KeyError:
            names = cat['recno']
        for an_x, an_y, name in zip(cat['x'], cat['y'], names):
            canvas.add(Circle(an_x, an_y, radius=10, color=color))
            canvas.add(Text(an_x + 10, an_y + 10 + offset, text=str(name), color=color))

catalog.observe(overlay_catalog, names='value')

In [15]:
hb

A Jupyter Widget

In [16]:
ra, dec = (268.029349, 37.545377)
coord = SkyCoord(ra, dec, unit='deg')
cat = cats[catalog.value[0]]
cat_coords = SkyCoord(cat['RAJ2000'], cat['DEJ2000'])
idx, _, _ = coord.match_to_catalog_sky(cat_coords)

IndexError: tuple index out of range

In [ ]:
ds = cat[idx]

In [ ]:
str(ds)

In [ ]:

cat, x, y = catalog_search(img.wcs.wcs, img.shape, 'B/vsx/vsx', radius=1, padding=0)

In [ ]:
cat

In [ ]:
x